In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns

In [26]:
from datetime import datetime
from hijri_converter import convert

def hijri(x):
    war = datetime.strptime(str(x)[0:10], '%Y-%m-%d')
    war1 = convert.Gregorian.fromdate(war).to_hijri()
    return war1

In [27]:
x='2032-08-03'
war = datetime.strptime(str(x)[0:10], '%Y-%m-%d')
war1 = convert.Gregorian.fromdate(war).to_hijri()
war1

Hijri(1454, 4, 26)

In [41]:
data = pd.read_csv("KSA Peak Analysis Timeline (Events Matrix).xlsx - Sheet1.csv")
data["Event"].unique()
data['Date']=pd.DatetimeIndex(data['Date'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_19156\3781886530.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Date']=pd.DatetimeIndex(data['Date'])


In [47]:
def get_future_dates(start_date, into_future, offset=1):

    start_date = start_date + np.timedelta64(offset,'D')
    end_date = start_date + np.timedelta64(into_future,'D')
    return np.arange(start_date, end_date, dtype='datetime64[D]')

In [64]:
df1=data
df1.index=pd.DatetimeIndex(df1['Date'])
start=df1.index[-1]+np.timedelta64(28,'D')


In [65]:
df1=data
df1.index=pd.DatetimeIndex(df1['Date'])
df1.index[-1]
next_time_steps = get_future_dates(start, 4138)
next_time_steps

array(['2021-09-03', '2021-09-04', '2021-09-05', ..., '2032-12-29',
       '2032-12-30', '2032-12-31'], dtype='datetime64[D]')

In [67]:
future=pd.DataFrame({'Date':next_time_steps})
future['hijri']=future['Date'].apply(hijri)
future['month_h']=future['hijri'].apply(lambda x:x.month)
future['day_h']=future['hijri'].apply(lambda x:x.day)
future

,Date,hijri,month_h,day_h
0,2021-09-03,1443-01-26,1,26
1,2021-09-04,1443-01-27,1,27
2,2021-09-05,1443-01-28,1,28
3,2021-09-06,1443-01-29,1,29
4,2021-09-07,1443-01-30,1,30
...,...,...,...,...
4133,2032-12-27,1454-09-24,9,24
4134,2032-12-28,1454-09-25,9,25
4135,2032-12-29,1454-09-26,9,26
4136,2032-12-30,1454-09-27,9,27


In [68]:
def haj(month,day):
    if month==12 and day>=8 and day<=13:
        return 1
    else : return 0

In [69]:
def ramadan(month):
    if month==9:
        return 1
    else : return 0

In [71]:
future['haj'] = future.apply(lambda x: haj(x['month_h'],x['day_h']), axis=1)
future[future['month_h']==12]

,Date,hijri,month_h,day_h,haj
300,2022-06-30,1443-12-01,12,1,0
301,2022-07-01,1443-12-02,12,2,0
302,2022-07-02,1443-12-03,12,3,0
303,2022-07-03,1443-12-04,12,4,0
304,2022-07-04,1443-12-05,12,5,0
...,...,...,...,...,...
3868,2032-04-06,1453-12-25,12,25,0
3869,2032-04-07,1453-12-26,12,26,0
3870,2032-04-08,1453-12-27,12,27,0
3871,2032-04-09,1453-12-28,12,28,0


In [73]:
future['ramdan'] = future['month_h'].apply(ramadan)

In [76]:
future=future[['Date','haj','ramdan']]
future.to_csv('event.csv',index=False)